In [18]:
import numpy as np

def generate_markov_chain_sequence(initial_state, transition_matrix, steps):
    current_state = initial_state
    sequence = [current_state]

    for _ in range(steps - 1):
        current_state = np.random.choice(['normal', 'depressed', 'manic'], p=list(transition_matrix[current_state].values()))
        sequence.append(current_state)

    return sequence

def simulate_population(population_size, steps, transition_matrix):
    population = []
    for _ in range(population_size):
        initial_state = 'normal' 
        sequence = generate_markov_chain_sequence(initial_state, transition_matrix, steps)
        population.append(sequence)

    return population

def categorize_mental_health_state(markov_chain_output):
    consecutive_depression = 0
    has_mania = False
    has_two_consecutive_depression = False

    for state in markov_chain_output:
        if state == 'depressed':
            consecutive_depression += 1
            if consecutive_depression >= 2:
                has_two_consecutive_depression = True
        elif state == 'manic':
            has_mania = True
            consecutive_depression = 0  
        else:
            consecutive_depression = 0  

    if has_two_consecutive_depression and has_mania:
        return 'Mixed Bipolar'
    elif has_two_consecutive_depression:
        return 'MDD'
    elif has_mania:
        return 'Pure Mania'
    else:
        return 'Normal'

def analyze_population_distribution(population):
    distribution = {'Normal': 0, 'MDD': 0, 'Mixed Bipolar': 0, 'Pure Mania': 0}

    for sequence in population:
        category = categorize_mental_health_state(sequence)
        distribution[category] += 1

    return distribution

transition_matrix = {'normal': {'normal': 0.9948432513018938, 'depressed': 0.005156748698106229, 'manic': 0.0}, 'depressed': {'normal': 0.5323753275500882, 'depressed': 0.4476687662528254, 'manic': 0.019955906197086426}, 'manic': {'normal': 0.2789316598230362, 'depressed': 0.45047841798780647, 'manic': 0.2705899221891573}}


population_size = 100000
sequence_length = 52  # 52 weeks
num_simulations = 1

sum_percentages = {'Normal': 0, 'MDD': 0, 'Mixed Bipolar': 0, 'Pure Mania': 0}

for _ in range(num_simulations):
    simulated_population = simulate_population(population_size, sequence_length, transition_matrix)
    population_distribution = analyze_population_distribution(simulated_population)
    population_percentages = {k: (v / population_size) * 100 for k, v in population_distribution.items()}
    for category in sum_percentages:
        sum_percentages[category] += population_percentages[category]
average_percentages = {k: v / num_simulations for k, v in sum_percentages.items()}

average_percentages

{'Normal': 88.663,
 'MDD': 10.405000000000001,
 'Mixed Bipolar': 0.594,
 'Pure Mania': 0.338}

In [15]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

def generate_markov_chain_sequence(initial_state, transition_matrix, steps):
    current_state = initial_state
    sequence = [current_state]
    for _ in range(steps - 1):
        current_state = np.random.choice(['normal', 'depressed', 'manic'], p=list(transition_matrix[current_state].values()))
        sequence.append(current_state)
    return sequence

def simulate_population(population_size, steps, transition_matrix):
    population = []
    for _ in range(population_size):
        initial_state = 'normal'
        sequence = generate_markov_chain_sequence(initial_state, transition_matrix, steps)
        population.append(sequence)
    return population

def categorize_mental_health_state(markov_chain_output):
    consecutive_depression = 0
    has_mania = False
    has_two_consecutive_depression = False
    for state in markov_chain_output:
        if state == 'depressed':
            consecutive_depression += 1
            if consecutive_depression >= 2:
                has_two_consecutive_depression = True
        elif state == 'manic':
            has_mania = True
            consecutive_depression = 0  
        else:
            consecutive_depression = 0  
    if has_two_consecutive_depression and has_mania:
        return 'Mixed Bipolar'
    elif has_two_consecutive_depression:
        return 'MDD'
    elif has_mania:
        return 'Pure Mania'
    else:
        return 'Normal'

def analyze_population_distribution(population):
    distribution = {'Normal': 0, 'MDD': 0, 'Mixed Bipolar': 0, 'Pure Mania': 0}
    for sequence in population:
        category = categorize_mental_health_state(sequence)
        distribution[category] += 1
    return distribution


target_distribution = {'Normal': 89, 'MDD': 10, 'Mixed Bipolar': 0.99999, 'Pure Mania': 0.00001}

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def generate_individual():
    return [random.random() for _ in range(9)]

toolbox.register("individual", tools.initIterate, creator.Individual, generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def normalize_probabilities(individual):
    for i in range(0, len(individual), 3):
        total = sum(max(0, p) for p in individual[i:i+3])  
        if total > 0:
            for j in range(3):
                individual[i+j] = max(0, individual[i+j]) / total
        else:
            individual[i:i+3] = [1/3, 1/3, 1/3]  
    return individual

def mutate(individual):
    for i in range(len(individual)):
        if random.random() < 0.1:  
            individual[i] += random.gauss(0, 0.1) 
            individual[i] = max(0, individual[i])  
    return individual,

toolbox.register("mutate", mutate)
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("select", tools.selTournament, tournsize=3)


def evaluate(individual):
    individual = normalize_probabilities(individual)
    transition_matrix = {
        'normal': {'normal': individual[0], 'depressed': individual[1], 'manic': individual[2]},
        'depressed': {'normal': individual[3], 'depressed': individual[4], 'manic': individual[5]},
        'manic': {'normal': individual[6], 'depressed': individual[7], 'manic': individual[8]}
    }
    simulated_population = simulate_population(1000, 52, transition_matrix)
    population_distribution = analyze_population_distribution(simulated_population)
    population_percentages = {k: (v / 1000) * 100 for k, v in population_distribution.items()}
    fitness = -sum(abs(population_percentages[k] - target_distribution[k]) for k in target_distribution)
    return fitness,

toolbox.register("evaluate", evaluate)


population_size = 300
num_generations = 50

population = toolbox.population(n=population_size)

for gen in range(num_generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))


best_ind = tools.selBest(population, 1)[0]
best_transition_matrix = {
    'normal': {'normal': best_ind[0], 'depressed': best_ind[1], 'manic': best_ind[2]},
    'depressed': {'normal': best_ind[3], 'depressed': best_ind[4], 'manic': best_ind[5]},
    'manic': {'normal': best_ind[6], 'depressed': best_ind[7], 'manic': best_ind[8]}
}

print("Best Transition Matrix:", best_transition_matrix)


Best Transition Matrix: {'normal': {'normal': 0.9678040009994667, 'depressed': 0.03219599900053324, 'manic': 0.0}, 'depressed': {'normal': 0.9288971783698942, 'depressed': 0.0699897555460989, 'manic': 0.0011130660840068427}, 'manic': {'normal': 0.2380042603750424, 'depressed': 0.52076819867353, 'manic': 0.24122754095142754}}
